<div style="display: flex; justify-content: center;">
    <img src="00_Logo_Finanzas_VLR_Verde_y_negro.png" alt="Texto alternativo" width="400"/>
</div>

#  Conectar Notebook de Jupyter (Python) a Datamart SQL de Power BI

## 1.  Configuración de librerías y paquetes (no hagas nada aquí)

Instalamos las librerías o paquetes tradicionales y para conexiones a base de datos

In [1]:
#Librería o paquetes estándar
import pandas as pd                                             # Importar pandas para manipulación de datos
import msal                                                     # Importar msal para autenticación
from sqlalchemy import create_engine                            # Importar create_engine para conexión a base de datos
from urllib.parse import quote_plus                             # Importar quote_plus para codificar la contraseña

##  2.  Conexión a DataMart Microsoft

-  Al contar con licenciamiento de Power BI Premium por usuario, se construyó un DataMart, el cual permite realizar el proceso ETL de las variables a utilizar en el presente modelo.  
-  La ventaja de este tipo de licenciamiento, es que su funcionamiento permite generar consultas SQL por un pago periódico mensual de 20 USD (feb 2025) y con capacidad máxima de 100 GB por modelo de datos: capacidad mas que suficiente para el presente proyecto.
-  Al contar con permisos de administración de Azure Active Directory se configuró conexión para lograr desarrollar en cuadernos de jupyter.  Dicha configuración incluyó validar temas de client_id, tenant_id, app, token, servidor, database, driver, entre otros. 

### 2.1.   Configura tu Client ID y tu Tenant ID

In [ ]:
# Configura los detalles de la aplicación AADa
client_id = 'xxxx'  # Tu Client ID registrado en AAD (Azure Active Directory)
tenant_id = 'xxxx'  # Tu Tenant ID en AAD (Azure Active Directory)

### 2.2. Configuracion de conexión (no hagas nada aquí)

In [3]:
authority = f'https://login.microsoftonline.com/{tenant_id}'

# Scope para acceder a Power BI API (Datamarts)
scope = ["https://analysis.windows.net/powerbi/api/.default"]

# Autentica y obtiene el token de acceso usando MSAL (Microsoft Authentication Library)
app = msal.PublicClientApplication(client_id, authority=authority)
token_response = app.acquire_token_interactive(scopes=scope)

if 'access_token' in token_response:
    access_token = token_response['access_token']
else:
    raise ValueError("No se pudo obtener el token de acceso. Verifica las credenciales y la configuración de la aplicación.")

### 2.3.  Configura tu cadena de conexión de datamart

In [ ]:
# Configura la conexión a la base de datos en Power BI Datamart
server = 'xxxx' #La cadena de conexión de tu datamart

### 2.4.  Configuración de conexión (no hagas nada aquí)

In [5]:
database = 'datamart' #Dejar *Datamart* por defecto
driver = 'ODBC Driver 17 for SQL Server'

# Crear cadena de conexión ODBC con Authentication explícita
conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Authentication=ActiveDirectoryInteractive;"
    f"AccessToken={access_token};"
)

# Codificar la cadena de conexión para SQLAlchemy
conn_str = quote_plus(conn_str)
connection_url = f"mssql+pyodbc:///?odbc_connect={conn_str}"

# Cargar el motor SQLAlchemy
engine = create_engine(connection_url)

## 3.  EDA (Exploratoy Data Analysis)

In [6]:
# Consulta SQL para obtener la tasa efectiva promedio ponderada de las colocaciones y los desembolsos por entidad financiera y por mes.
query = """
SELECT * 
FROM
    Consulta
 """

In [7]:
# Ejecuta la consulta y carga los datos en un DataFrame de pandas
df = pd.read_sql_query(query, engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835 entries, 0 to 834
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fecha                  835 non-null    object 
 1   indice                 835 non-null    float64
 2   variacion_anual        835 non-null    float64
 3   variacion_mensual      835 non-null    float64
 4   variacion_ano_corrido  835 non-null    float64
dtypes: float64(4), object(1)
memory usage: 32.7+ KB


In [8]:
df.tail(15)

,fecha,indice,variacion_anual,variacion_mensual,variacion_ano_corrido
820,2023-11-30,137.09,10.15,0.47,8.78
821,2023-12-31,137.72,9.28,0.45,9.28
822,2024-01-31,138.98,8.35,0.92,0.92
823,2024-02-29,140.49,7.74,1.09,2.01
824,2024-03-31,141.48,7.36,0.70,2.73
825,2024-04-30,142.32,7.16,0.59,3.34
826,2024-05-31,142.92,7.16,0.43,3.78
827,2024-06-30,143.38,7.18,0.32,4.12
828,2024-07-31,143.67,6.86,0.20,4.32
829,2024-08-31,143.67,6.12,0.00,4.33
